In [2]:
# Bow Bag of Words CountVectorizer
# 문서를 고정된 길이의 벡터로 변환
# 문서 - 단어행렬
# 장점 : 간단하고 빠름
# 단점 : 단어 순서 손실, 희소성, 의미적 유사성 무시

# tf-idf TfidfVectorizer
# 모든 문서에서 자주 등장하는 단어의 영향을 줄이고 문서 특이 단어를 강조

# multinomal Navie Bayes 확률 모델
# LogisticRegression 다중 클래스 회귀 기반 분류

# RidgeClassifier 회귀 기반 분류 L2 규제

# N-gram 단점 : 차원 폭발에 주의 (정규화/차원 축소 고려)

# Kolnpy Okt

In [3]:
import numpy as np
import pandas as pd
# scikit-learn
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
# 분류모델
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [4]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

# 데이터 로드
newsgroup_train = fetch_20newsgroups(subset='train',
                remove=('headers','footers','quotes'),
                categories=categories
                )
newsgroup_test = fetch_20newsgroups(subset='test',
                remove=('headers','footers','quotes'),
                categories=categories
                )

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
- alt.atheism 무신론, 종교 비판 토론 종교의 존재, 신의 존재 유무, 종교적 주장 반박, 철학적 논쟁 등
- talk.religion.misc 일반 종교 토론 (기타 잡담 포함) 기독교, 불교, 이슬람 등 다양한 종교 관련 이야기, 개인 경험, 신념 공유 등
- comp.graphics 컴퓨터 그래픽스, 이미지 처리 3D 렌더링, 이미지 파일 포맷, 그래픽 소프트웨어 사용법, OpenGL 등 기술 관련 토론
- sci.space 우주 과학, 천문학 로켓, NASA, 행성 탐사, 외계 생명 가능성, 우주 물리학 관련 토론

In [7]:

from sklearn.datasets import load_files
train_path = r'C:\\Users\\sally\\SKNPYWORKSPACE\\LLM\\ch05\\20newsbydate\\20news-bydate-train'
test_path = r'C:\\Users\\sally\\SKNPYWORKSPACE\\LLM\\ch05\\20newsbydate\\20news-bydate-test'
newsgroups_train = load_files(train_path,encoding='latin1')
newsgroups_test = load_files(test_path,encoding='latin1')

In [8]:
categories

['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

In [9]:
# 카테고리 제거
def filter_categories(dataset, categories):
    target_names = dataset.target_names
    selected_idx = [ target_names.index(c) for c in categories  ]
    #필터링
    data_filtered, target_filtered = [], []
    for text,label in zip(dataset.data, dataset.target):
        if label in selected_idx:
            new_label = selected_idx.index(label)  # 라벨 재 정렬
            data_filtered.append(text) ; target_filtered.append( new_label  )
    return data_filtered,target_filtered,categories

In [10]:
train_data, train_target, target_names = filter_categories(newsgroups_train,categories)
test_data, test_target, _ = filter_categories(newsgroups_train,categories)

In [11]:
# 해더 푸터 인용문 제거

In [12]:
import re

def clean_text(text):
    # 헤더 제거
    text = re.sub(r'^From:.*\n', '', text, flags=re.MULTILINE)
    text = re.sub(r'^Subject:.*\n', '', text, flags=re.MULTILINE)

    # 풋터 제거
    text = re.sub(r'\n--\n.*$', '', text, flags=re.DOTALL)

    # 인용문 제거
    text = re.sub(r'(^|\n)[>|:].*', '', text)

    return text

In [13]:
train_data = [ clean_text(t) for t in train_data]
test_data = [ clean_text(t) for t in test_data]

In [14]:
len(train_data), len(train_target), len(test_data), len(test_target)

(2034, 2034, 2034, 2034)

# 멀티노멀 나이브베이즈 (Multinomial Naive Bayes)

- 문서에 포함된 단어들의 출현 확률을 기반으로 해서 그 문서가 어떤 주제에 속할지 확률적으로 계산  
- **스팸 필터링**, **뉴스 기사 카테고리 분류**, **감성 분석** 등에 사용  
- **베이즈 정리 확률 이론** 기반 → 조건부 확률  
- **단어가 나왔을 때 이 문서가 스팸일 확률**을 계산  

\[
P(\text{스팸}|\text{단어들}) = \frac{P(\text{단어들}|\text{스팸}) \times P(\text{스팸})}{P(\text{단어들})}
\]

---

## 🌱 나이브 (Naive): 순진한 가정

- **가정**: 문서 안의 모든 단어는 서로 독립적이다.  
- **현실**: 스팸에 자주 나오는 단어들은 서로 독립적이지 않다.  
- **실제**: 이러한 가정은 계산량을 빠르게 하고 단순하지만, 어느 정도 정확도를 보장한다.  

---

## 💡 멀티노멀: 다항 분포 기반

- **의미**: 단어의 출현 **횟수**를 중요하게 본다.  
- 횟수를 세는 멀티노미얼 방식이 NLP에 잘 맞는다.  
- 모델은 단어의 **빈도수 통계**를 사용한다.  

---

### 📊 스팸 메일 통계 (spam)

| 단어 | 빈도 |
|------|------|
| free | 150  |
| money | 100 |
| viagra | 50 |
| report | 5  |

---

### 📊 정상 메일 통계 (ham)

| 단어 | 빈도 |
|------|------|
| report | 80 |
| meeting | 60 |
| free | 10  |

---

## 📈 확률 계산

이러한 통계를 바탕으로, 특정 카테고리(예: 스팸)에서 단어가 나올 확률을 계산한다.

\[
P(\text{‘free’}|\text{스팸})
\]

예를 들어 문서가 `"Free money meeting"`일 때:

- 스팸일 확률  
  \[
  P(\text{스팸}) \times P(\text{free}|\text{스팸}) \times P(\text{money}|\text{스팸}) \times P(\text{meeting}|\text{스팸})
  \]

- 정상일 확률  
  \[
  P(\text{정상}) \times P(\text{free}|\text{정상}) \times P(\text{money}|\text{정상}) \times P(\text{meeting}|\text{정상})
  \]

이 두 값을 비교하여 더 큰 확률을 가진 쪽으로 분류한다.


In [15]:
# nltk tokenizer stemer
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sally\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

In [17]:
# min_df : 단어의 빈도가 최소 5개의 문서에 등장  - 노이즈 감소
# max_df : 50% 너무흔한 단어는 제거
cv = CountVectorizer(max_features=2000,min_df=5, max_df=0.5)
x_train_cv = cv.fit_transform(train_data)
x_test_cv = cv.transform(test_data)
x_train_cv.shape,  x_test_cv.shape

((2034, 2000), (2034, 2000))

In [18]:
cv.get_feature_names_out()

array(['00', '000', '01', ..., 'zip', 'zoo', 'zoology'],
      shape=(2000,), dtype=object)

In [19]:
(x_train_cv[0].toarray()[0])[x_train_cv[0].toarray()[0]>0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1])

In [22]:
# Bow 기반
# 텍스트 분류의 강력한 baseline 희소데이터에 강함
# 모델선택
nb = MultinomialNB()
# 학습용 데이터 벡터 데이터
nb.fit(x_train_cv, train_target)

print(nb.score(x_train_cv, train_target), nb.score(x_test_cv,test_target))

# 분류 리포트
from sklearn.metrics import classification_report
y_pred_nb = nb.predict(x_test_cv)
print(classification_report(test_target,y_pred_nb, target_names=categories))

0.9090462143559489 0.9090462143559489
                    precision    recall  f1-score   support

       alt.atheism       0.90      0.90      0.90       480
talk.religion.misc       0.87      0.83      0.85       377
     comp.graphics       0.92      0.96      0.94       584
         sci.space       0.93      0.92      0.93       593

          accuracy                           0.91      2034
         macro avg       0.90      0.90      0.90      2034
      weighted avg       0.91      0.91      0.91      2034



In [23]:
# TF_IDF + MNB + LogisticRegression
# TF-IDF 기반으로 중요단어 강조, 선형모델과 자주사용, BOW 대비 흔한 단어 영향 감소
tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df = 0.5)
x_train_tfid = tfidf.fit_transform(train_data)
x_test_tfid = tfidf.transform(test_data)

# NB + tf-idf
nb_tfidf = MultinomialNB()
nb_tfidf.fit(x_train_tfid, train_target)
print(nb_tfidf.score(x_train_tfid, train_target), nb_tfidf.score(x_test_tfid, test_target))


0.9287118977384464 0.9287118977384464


In [32]:
# logistic 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test \
    = train_test_split(x_train_tfid,train_target,test_size=0.2
                       ,stratify=train_target,random_state=42)
lr = LogisticRegression(max_iter=1000)
lr.fit(x_train, y_train)
print(lr.score(x_train,y_train),  lr.score(x_test, y_test) )

0.9704978488014752 0.8894348894348895


In [33]:
# 과적합 해결을 위한 규제
rc = RidgeClassifier(alpha=10)
rc.fit(x_train, y_train)
print(rc.score(x_train,y_train),  rc.score(x_test, y_test) )

0.9317762753534112 0.8722358722358723


In [34]:
# L1 규제   L1 Logistic(Lasso와 유사)
# 일부 계수를 0으로 만들어서 특성 선택을 수행.. 중요피처 select 효과
l1_lr = LogisticRegression(penalty='l1',max_iter=1000,solver='saga')
l1_lr.fit(x_train,y_train)
print(l1_lr.score(x_train,y_train),  l1_lr.score(x_test, y_test) )

0.8893669330055316 0.8329238329238329


In [36]:
# 트리모델 + tfidf
tree = DecisionTreeClassifier(max_depth=3)
fores = RandomForestClassifier(max_depth=3)
dg = GradientBoostingClassifier(max_depth=3)

In [37]:
gb = GradientBoostingClassifier(max_depth=2)
gb.fit(x_train,y_train)
print(gb.score(x_train,y_train),  gb.score(x_test, y_test) )

0.9477566072526121 0.828009828009828


In [38]:
x_train,x_test,y_train,y_test = train_test_split(train_data,train_target
         ,stratify=train_target,test_size=0.2,random_state=42)

In [ ]:
# 전처리 
# RegexpTokenizer + stopwords + PorterStemmer
english_stops = set(stopwords.words('english'))
regtok = RegexpTokenizer(r"[\w']{3,}")
def custom_tokenizer(text):
    toks = regtok.tokenize(text.lower())
    toks = [t for t in toks if t not in english_stops]
    toks = [PorterStemmer().stem(t) for t in toks]
    return toks
tfidf_custom = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=2000,min_df=5, max_df=0.5)
x_train_tfidf_c = tfidf_custom.fit_transform(x_train)
x_test_tfidf_c = tfidf_custom.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_c = LogisticRegression(max_iter=1000)
lr_c.fit(x_train_tfidf_c,y_train)
print( lr_c.score(x_train_tfidf_c, y_train) , lr_c.score(x_test_tfidf_c, y_test))

In [ ]:
# n-gram 실험 1,2  1,3
# 성능향상 기대  연속된 단어패턴 포착
tfidf_12 = TfidfVectorizer(token_pattern = r"[\w']{3,}"
                           ,stop_words= stopwords.words('english')
                           ,ngram_range=(1,2)
                           , min_df=2, max_df=0.5
                        #    ,max_features=2000
                           )
x_train_12 = tfidf_12.fit_transform(x_train)
x_test_12 = tfidf_12.transform(x_test)

lr_c = LogisticRegression(max_iter=1000)
lr_c.fit(x_train_12,y_train)
print( lr_c.score(x_train_12, y_train) , lr_c.score(x_test_12, y_test))

In [ ]:
# 한국어 처리  konlpy
# 품사기반 태깅 tokenizer   Noun Verb Adjetive
# 데이터 로딩
import pandas as pd
url = "https://drive.google.com/uc?id=1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o"
df = pd.read_csv(url)
df.head()

In [ ]:
df.title.unique()

In [ ]:

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test =  train_test_split(df.review
    ,df.title,stratify=df.title, test_size=0.2,random_state=42)

In [ ]:

from konlpy.tag import Okt
okt = Okt()

In [ ]:
# simple version
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)
clf = LogisticRegression(max_iter=1000)

In [ ]:

clf.fit(x_train_tfidf,y_train)

In [ ]:
x_train_tfidf.shape  , y_train.shape

In [ ]:
clf.score(x_train_tfidf,y_train), clf.score(x_test_tfidf,y_test)

In [ ]:

def custom_tokenizer(text):
    target = ['Noun','Verb','Adjective']
    return [w for w,tag in okt.pos(text,norm= True, stem=True) if tag in target]

tfidf = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=2000, min_df=5, max_df=0.5)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)
clf = LogisticRegression(max_iter=1000)
clf.fit(x_train_tfidf,y_train)
clf.score(x_train_tfidf,y_train), clf.score(x_test_tfidf,y_test)